## apply,map함수  
 1.apply   
   1) series, dataframe객체 모두 사용 가능  
  ```python
    #단일인자
    def ex_year(col):
        return col.split('-')[0]
    df['bDate'].apply(ex_year)  
    
    #multi 인자, 첫값 선택 
    def funcA(x,y):
        return x,y   #리스트 리턴
    df.apply(lambda x:funcA(x['cola'],x['colB'][0]) 
             
  ```    
 2.map  
   1) apply와 같은 방식으로 사용가능하나 series객체만 사용 가능  
      따라서, 인자도 1개만 가능함 -> 복수 인자는 apply사용해야  
   2) 아래처럼 map함수내에 dict를 전달하여 값 변환 가능    
      df.name.map({'서울':'1','부산':'2','대전':'3'})   

In [ ]:
import pandas as pd
dict=[ {'name':'april','cls':'a','bDate':'2018-01-17','salary':300, 'amt':30000.10},
       {'name':'babe','cls':'a','bDate':'2018-01-22','salary':400, 'amt':33000.1088},
       {'name':'cyan','cls':'b','bDate':'2018-05-08','salary':500, 'amt':36000.1088},
       {'name':'cyan','cls':'b','bDate':'2015-11-03','salary':500, 'amt':36000.1077},
       {'name':'eric','cls':'c','bDate':'2018-07-10','salary':500, 'amt':36000.1678},
       {'name':'gag','cls':'b','bDate':'2015-01-02','salary':500, 'amt':36000.10},
       {'name':'little','cls':'d','bDate':'2013-11-02','salary':500, 'amt':36000.15},
       {'name':'doct','cls':'a','bDate':'2010-01-02','salary':600, 'amt':56000.1550}]

df=pd.DataFrame(dict)

In [ ]:
df.name.map({'cyan':'1','doct':'2','':''}) # map으로 각 항목값을 변환

In [ ]:
def ex_year(col):
    return col.split('-')[0]

def ex_age(bdate, cdate):
    return int(cdate.split('-')[0]) - int(bdate.split('-')[0])

In [ ]:
df['bDate'].apply(ex_age, cdate='2020-02-01')

### group by ,count  
1.groupby  
2.pivot  
 -column을 row로 변경  

In [ ]:
df.name.value_counts() #name 각각에 대한 건수

In [ ]:
df.name.unique()  #unique한 name리스트

In [ ]:
groupby_class=df.groupby('cls') #그룹 object 생성
groupby_class.groups
for name,group in groupby_class:
    print(name +' : ' + str(len(group)))  #그룹별 건수
    print(group)  #그룹화된 df 출력
    print(' ')

In [ ]:
df_class_cnt=pd.DataFrame({'count':groupby_class.size()})  #데이터프레임으로

In [ ]:
df_class_cnt=pd.DataFrame({'count':groupby_class.size()}).reset_index  #index을 컬럼이름으로 

In [ ]:
df[df.cls.isin(['a','b'])]

In [ ]:
df.name.value_counts()

In [ ]:
df.pivot(columns='name',values='salary')

1. 결측치보정, join (코드변경), groupby, merge, reshape column
2. join  

In [ ]:
import pandas as pd
fl='D:/글로벌/실적분석/곰스_2002.xlsx'
#fl='E:/글로벌백업(180329)/04.수수료작업/201912수수료/계약목록.xlsx'
df= pd.read_excel(fl)

df= df[df.NO!='합계']   #불필요 데이터 삭제
df['지점'].fillna('엠피메트로', inplace=True)  #결측치 보정

#지점명 변환 join
#fl='D:/글로벌/실적분석/코드파일.xlsx'
fl='E:/글로벌백업(180329)/04.수수료작업/코드목록.xlsx'
df_org= pd.read_excel(fl,sheet_name='지점변환')
df=df.join(df_org.set_index('수금사원명')['변환지점'], on='수금사원명')  
df['변환지점']=df['변환지점'].fillna(df.지점)
#컬럼 rename
df=df.rename(columns={'지점':'원지점','변환지점':'지점'}) 

In [ ]:
#그룹by 집계
grp=['지점','수금사원명','계약종류']

grpval='초회보험료'
df_l=pd.DataFrame(df[df.계약종류=='장기'].groupby(grp)[grpval].agg('sum')).sort_values(grp, ascending=False)
df_s=pd.DataFrame(df[df.계약종류=='생보'].groupby(grp)[grpval].agg('sum')).sort_values(grp, ascending=False)

In [ ]:
df_l.reset_index(inplace=True)  # index를 컬럼으로
df_s.reset_index(inplace=True)
joins=['지점','수금사원명']
df_m=pd.merge(df_l,df_s,how='outer',on=joins) #컬럼merge ([[안에 필요컬럼만 지정시 아래 필요없음]])
#불필요 컬럼삭제, 이름변경
df_m=df_m.fillna(0).drop(columns=['계약종류_x','계약종류_y']).rename(columns={'초회보험료_x':'장기','초회보험료_y':'생보'})
df_m['합계']=df_m.장기+df_m.생보

#### 1)지점그룹값을 다른 df에서 읽어오는 방법(1)  
    - 코드파일읽기 -> join으로 '지점그룹' 컬럼 생성
    - 결과를 excel로

In [ ]:
#지점그룹을 만들기 : 코드파일읽어 join
fl='D:/글로벌/실적분석/코드파일.xlsx'
#fl='E:/글로벌백업(180329)/04.수수료작업/코드목록.xlsx'
df_grp= pd.read_excel(fl,sheet_name='지점그룹')
df_m=df_m.join(df_grp.set_index('지점')['지점그룹'], on='지점') 

In [ ]:
df_m[df_m.지점그룹=='직영'].sort_values('합계',ascending=False).head(2)
df_m[df_m.지점그룹=='직영'].sort_values('합계',ascending=False).tail(2)

In [ ]:
with pd.ExcelWriter('E:/글로벌백업(180329)/04.수수료작업/집계.xlsx') as writer:  
    df_m.to_excel(writer, sheet_name='지점별',index=False)

In [ ]:
# 회사별 집계

In [ ]:
df_co = df[df.계약종류.isin(['장기','생보'])].groupby(['계약종류','보험사','지점'])['초회보험료'].agg(['sum','count'])

In [ ]:
df_co.pivot_table(index=['계약종류','보험사'],aggfunc='sum',values='sum')

In [ ]:
with pd.ExcelWriter('D:/글로벌/실적분석/실적요약.xlsx') as writer:  
    df_m.to_excel(writer, sheet_name='지점별',index=False)
    df_co.to_excel(writer, sheet_name='회사별',merge_cells=False,float_format='%f')

#### 2) 지점그룹값을 읽어오는 다른 방법(2)    
   - 코드파일읽기 -> indexing -> 컬럼생성 (이때 index가 같은 series이므로 pandas가 알아서 매칭해주는 원리임)    

In [ ]:
#머지의 다른 방법
fl='D:/글로벌/실적분석/코드파일.xlsx'
#fl='E:/글로벌백업(180329)/04.수수료작업/코드목록.xlsx'
df_grp= pd.read_excel(fl,sheet_name='지점그룹')
df_m2=df_m
df_m2.set_index('지점',inplace=True)
df_grp.set_index('지점',inplace=True)
df_m2['지점그룹']=df_grp['지점그룹']  ## df_m2와 df_grp에 같은 컬럼으로 인덱스가 있는 경우
df_m2.reset_index()
df_grp.reset_index()

### df에 대한 for loop  
1.iterrows 이용  
  속도가 느려 비추천이지만, 꼭 필요할 때 사용  
2.apply를 이용  
  row단위로 처리하는 함수를 만들어 두고 apply를 이용하여 전체 호출

In [ ]:
for seq, row in df.iterrows():  #DF의 각 행을 LOOP
        row['합']=row['국어']+row['수학']++row['영어']
        print(seq,'=',list(row))     
        if row['국어'] > 80:
            print('우수')
            
''' 위 코드를 한줄로       
 df['합계']=df.국어 + df.영어 + df.수학
'''

def grade(row):
     row['합']=row['국어']+row['수학']++row['영어']
     if row['국어'] > 80:
        row['국어등급']='우수'
     return row

#ss=list(filter(lambda x: (x>80),df['국어'])) 
df2=df.apply(grade,axis=1)
df3=df.apply

grades = []

for row in df['국어']:
    if row >= 90:        grades.append('A')
    elif row >= 80:      grades.append('B')
    elif row >= 70:      grades.append('C')
    else:                grades.append('F')
        
df['grade'] = grades

In [ ]:
 #DF의 각 열을 LOOP  
for lab, cont in df.items(): 
      print(lab,'=',list(cont))

### lambda 사용  
 간단한 익명함수제작  
    lambda x,y:  x+y  [인수: 반환표현식]  
    lambda x: 'a' if x==1 else 'b' [인수: 조건표현식]  
    lambda x: 'a' if x==1 else ( 'b' if x==2 else 'c')  [elif필요시... 2개이상이면 복잡해지므로 함수 만들어 호출이 편리  

In [ ]:
ret=lambda x,y,z: x+y+z
ret(10,11,11)

def prt(val):
    if val >400:
       return '우수'
    else: return '미흡'   

In [ ]:
# df.salary * 80을 반환
df.salary.map(lambda x: x*80)
# lambda식내에서 함수 호출
df.salary.map(lambda x: (prt(x)))

# if문 간략히
df.salary.map(lamda x: '우수' if x<400 else '미흡')

In [ ]:
##함수만들어 호출
def getGrade(x):
    if x>400:      return 'A'
    elif x> 300:   return 'B'
    elif x> 200:   return 'C'
    else:          return 'D'
    
df.salary.apply(lambda x: getGrade(x) )

In [ ]:
s=lambda x,y: getGrade(x) 
s(200,300)

In [ ]:
# dict를 사용하여 1:1 변환
dict={'a':'A','b':'B','c':'C','d':'D'}
df['cls'].map(lambda x: dict[x])

##### 컬럼값 update  
1.replace사용  
-특정 조건에 맞는 값만 변환 가능  
2.map함수이용  
-모든 값을 변환시 가능  

In [ ]:
df.name=df.name.replace('april','may')

## 컬럼 조작
 -특정컬럼만 추출, 이동

In [ ]:
## 필요시 함수로 만들어 사용해도 됨
colsSelect = ['name','salary']  #대상 컬럼 지정
type='move'  #sel-추출, skip-제외, Fmove-알으로 , Bmove- 뒤로
cols=[]
if type=='incl': 
     cols= cols + [col for col in df if col in colsSelect] 
elif type=='excl': 
     cols= cols=cols + [col for col in df if col not in colsSelect] 
elif type=='Fmove': 
     cols= colsSelect + [col for col in df if col not in colsSelect] 
elif type=='Bmove': 
     cols= [col for col in df if col not in colsSelect] + colsSelect 
else : cols=df.columns
        
df[cols]

## 데이터 row 분할 by 범주형 컬럼

In [ ]:
#그룹by로 분할하여 dict로.. 필요에 따라 후 처리하면 됨

groupby_cls=df.groupby('cls')
#---1. dictionary에 담는 방법
dict = {} 
for name, group in groupby_cls:
    dict['분할'+name]= pd.DataFrame(groupby_cls.get_group(name))
    print(dict['분할'+name])

print(dict.keys())
for key in dict.keys():
    print(dict[key].shape)
    print(dict[key].columns)

#---2. list에 담는 방법    
dfList=[]
for name, group in groupby_cls:
    dfList.append( pd.DataFrame(groupby_cls.get_group(name)) )

print(len(dfList))

In [234]:
#--3. map함수를 이용하여 분할된 데이터 처리방법
def splitBy(grpData):
    print(grpData)
#   파일로 출력 등 후 처리 로직 넣으면 됨
    
list(map(splitBy,groupby_cls))

('a',     name cls       bDate  salary         amt    등급 등급2
0  april   a  2018-01-17     300  30000.1000   low   a
1   babe   a  2018-01-22     400  33000.1088   low   a
7   doct   a  2010-01-02     600  56000.1550  high   c)
('b',    name cls       bDate  salary         amt   등급 등급2
2  cyan   b  2018-05-08     500  36000.1088  mid   b
3  cyan   b  2015-11-03     500  36000.1077  mid   b
5   gag   b  2015-01-02     500  36000.1000  mid   b)
('c',    name cls       bDate  salary         amt   등급 등급2
4  eric   c  2018-07-10     500  36000.1678  mid   b)
('d',      name cls       bDate  salary       amt   등급 등급2
6  little   d  2013-11-02     500  36000.15  mid   b)


[None, None, None, None]

In [231]:
dfList[0]

,name,cls,bDate,salary,amt,등급,등급2
0,april,a,2018-01-17,300,30000.1000,low,a
1,babe,a,2018-01-22,400,33000.1088,low,a
7,doct,a,2010-01-02,600,56000.1550,high,c


## 데이터분할 by 분포 구간

In [2]:
#cut-숫자형으로 등급분할 컬럼 추가
gd=[0,300,500,900,1000]
lbl=['a','b','c','d']
df['등급']=pd.cut(df.salary,bins=gd,labels=lbl)

In [5]:
# qcut - 숫자형으로 등급분할 컬럼 추가 / 등급개수 지정방식임
lbl=['a','b','c']
df['등급2']=pd.qcut(df.salary,q=4,labels=lbl,duplicates='drop')

In [10]:
bins_val

array([300., 400., 500., 600.])

In [7]:
import numpy as np
bins_name=['low','mid','high']
count,bins_val=np.histogram(df.salary,bins=3) # 3분할
df['등급3']=pd.cut(df.salary, 
            bins=bins_val,  #경계값 리스트
            labels=bins_name,  #bin이름
            include_lowest=True) #첫 경계값 포함여부

## 비교
  - 문자열 포함   
    '찾는값' in (var)  #var에 찾는값 부분일치   
  - 리스트 포함  
    series.isin(list)  
  - null체크  
     pd.isnull(var)  
     pd.notnull(var)  
     if var==None  

In [15]:
import pandas as pd
lis=['홍길동전','이순신']
lis2=['홍길동']
ser=pd.Series(['홍길동전','이순신'])
str='홍길동전'
print('홍' in lis) # => false
print('홍' in str) # => True  
print(ser.isin(lis2)) #=> 
print(ser.isin(ser))  #=>
pd.notnull(str)

False
True
0    False
1    False
dtype: bool
0    True
1    True
dtype: bool


True